In [5]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm

### Read the data

In [6]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [10]:
class Brick:
    def __init__(self, brick_string, name):
        self.name = name
        end1, end2 = brick_string.split('~')
        self.x1, self.y1, self.z1 = [int(x) for x in end1.split(',')]
        self.x2, self.y2, self.z2 = [int(x) for x in end2.split(',')]

    @property
    def lowest_z(self):
        return min(self.z1, self.z2)
    
    @property
    def highest_z(self):
        return max(self.z1, self.z2)
    
    @property
    def lowest_y(self):
        return min(self.y1, self.y2)
    
    @property
    def highest_y(self):
        return max(self.y1, self.y2)
    
    @property
    def lowest_x(self):
        return min(self.x1, self.x2)
    
    @property
    def highest_x(self):
        return max(self.x1, self.x2)
    
    @property
    def on_ground(self):
        return self.lowest_z == 1
    
    def drop_one(self):
        self.z1 -= 1
        self.z2 -= 1

    def drop_n(self, n):
        self.z1 -= n
        self.z2 -= n
    
    def x_overlap(self, other):
        return self.lowest_x <= other.highest_x and self.highest_x >= other.lowest_x
    
    def y_overlap(self, other):
        return self.lowest_y <= other.highest_y and self.highest_y >= other.lowest_y

    def __repr__(self):
        return 'Brick {}: ({}, {}, {}) ~ ({}, {}, {})'.format(self.name, self.x1, self.y1, self.z1, self.x2, self.y2, self.z2)
    
    def __eq__(self, other):
        return self.x1 == other.x1 and self.y1 == other.y1 and self.z1 == other.z1 and self.x2 == other.x2 and self.y2 == other.y2 and self.z2 == other.z2


class BrickSpace:
    def __init__(self, working_file_content):

        self.bricks = [Brick(x, 'A') for x in working_file_content]

    def move(self):
        # sort bricks on lowest z value
        print("Moving bricks..")
        self.bricks.sort(key=lambda x: x.lowest_z, reverse=False)
        for brick in tqdm(self.bricks):
            distance_to_nearest_brick_below = self.get_distance_to_nearest_brick_below(brick)
            brick.drop_n(distance_to_nearest_brick_below)
            # while self.can_move(brick):
            #     brick.drop_one()

    def get_distance_to_nearest_brick_below(self, brick):
        nearest_overlapping_brick = None
        distance = 9999999999
        for other_brick in self.bricks:
            if brick == other_brick:
                continue
            if brick.x_overlap(other_brick) and brick.y_overlap(other_brick):
                if (brick.lowest_z - other_brick.highest_z) > 1 and (brick.lowest_z - other_brick.highest_z) < distance:
                    nearest_overlapping_brick = other_brick
                    distance = (brick.lowest_z - other_brick.highest_z)
        if nearest_overlapping_brick:
            return distance - 1
        return 0

    def can_move(self, brick):
        if brick.on_ground:
                return False
        lowest_z = brick.lowest_z
        for other_brick in self.bricks:
            # not current brick
            if brick == other_brick:
                continue
            # check if brick is directly below
            if other_brick.highest_z == lowest_z - 1 and brick.x_overlap(other_brick) and brick.y_overlap(other_brick):
                return False
        return True
    
    def supports(self, brick):
        other_bricks = list()
        highest_z = brick.highest_z
        for other_brick in self.bricks:
            # not current brick
            if brick == other_brick:
                continue
            # check if brick is directly above
            if other_brick.lowest_z == highest_z + 1 and brick.x_overlap(other_brick) and brick.y_overlap(other_brick):
                other_bricks.append(other_brick)
        return other_bricks
    
    def is_supported_by(self, brick):
        other_bricks = list()
        lowest_z = brick.lowest_z
        for other_brick in self.bricks:
            # not current brick
            if brick == other_brick:
                continue
            # check if brick is directly above
            if other_brick.lowest_z == lowest_z - 1 and brick.x_overlap(other_brick) and brick.y_overlap(other_brick):
                other_bricks.append(other_brick)
        return other_bricks
    
    def is_essential(self, brick):
        # a brick is essential if any brick that it supports is supported by only this one
        bricks_that_are_supported_by_this = self.supports(brick)
        if len(bricks_that_are_supported_by_this) == 0:
            return False

        for other_brick in bricks_that_are_supported_by_this:
            bricks_that_support_other_brick = self.is_supported_by(other_brick)
            if len(bricks_that_support_other_brick) == 1:
                return True
            
        return False
        
    def n_bricks_that_can_be_disintegrated(self):
        n = 0
        print("Checking number of disintegratable bricks..")
        for brick in tqdm(self.bricks):
            if not self.is_essential(brick):
                n += 1
        return n

    
    def __repr__(self):
        # sort bricks
        self.bricks.sort(key=lambda x: x.lowest_z, reverse=True)

        return_str = ''
        for brick in self.bricks:
            return_str += str(brick)
            if not self.is_essential(brick):
                return_str += ' can be disintegrated'
            return_str += '\n'
            
        return return_str

                
working_file_content = file_content

bricks = BrickSpace(working_file_content)
bricks.move()
print(bricks.n_bricks_that_can_be_disintegrated())

Moving bricks..


  0%|          | 0/1370 [00:00<?, ?it/s]

100%|██████████| 1370/1370 [00:03<00:00, 376.22it/s]


Checking number of disintegratable bricks..


100%|██████████| 1370/1370 [00:02<00:00, 648.66it/s]

511


### Part 2

In [16]:
infile = open("day22.txt")
#infile = open("sample.txt")

cubes = {}

for i, line in enumerate(infile):
    p0, p1 = (tuple(map(int, p.split(","))) for p in line.strip().split("~"))
    cubes[i] = (p0, p1)

gminx = min(min(x0, x1) for ((x0, _, _), (x1, _, _)) in cubes.values())
gminy = min(min(y0, y1) for ((_, y0, _), (_, y1, _)) in cubes.values())
gmaxx = max(max(x0, x1) for ((x0, _, _), (x1, _, _)) in cubes.values())
gmaxy = max(max(y0, y1) for ((_, y0, _), (_, y1, _)) in cubes.values())

grid = dict(((x,y,0), -1) for x in range(gminx, gmaxx+1) for y in range(gminy, gmaxy+1))

# lower cubes starting from the lowest ones, stopping when they collide with something
to_lower = list(range(len(cubes)))
to_lower.sort(key = lambda i: min(cubes[i][0][2], cubes[i][1][2]))
resting_on = {}
while to_lower:

    c_i = to_lower.pop(0)
    c = cubes[c_i]

    rest = set()
    rest_z = -1

    cminx = min(c[0][0], c[1][0])
    cminy = min(c[0][1], c[1][1])
    cminz = min(c[0][2], c[1][2])
    cmaxx = max(c[0][0], c[1][0])
    cmaxy = max(c[0][1], c[1][1])
    cmaxz = max(c[0][2], c[1][2])

    for ((gx, gy, gz), gi) in grid.items():
        if gx >= cminx and gx <= cmaxx and gy >= cminy and gy <= cmaxy:
            if gz > rest_z:
                rest_z = gz
                rest = set([gi])
            elif gz == rest_z:
                rest.add(gi)

    print(f"{c_i} resting on {rest}")
    resting_on[c_i] = rest

    new_z = rest_z + 1
    delta_z = new_z - cminz
    c = ((c[0][0], c[0][1], c[0][2] + delta_z),
         (c[1][0], c[1][1], c[1][2] + delta_z))
    cminz = min(c[0][2], c[1][2])
    cmaxz = max(c[0][2], c[1][2])

    for z in range(cminz, cmaxz+1):
        for y in range(cminy, cmaxy+1):
            for x in range(cminx, cmaxx+1):
                assert((x, y, z) not in grid)
                grid[(x, y, z)] = c_i

can_disintegrate = set(range(len(cubes)))
for rs in resting_on.values():
    if len(rs) == 1:
        ri, = rs
        can_disintegrate.discard(ri)

print(can_disintegrate)
print(len(can_disintegrate))

infile = open("day22.txt")
#infile = open("sample.txt")

cubes = {}

for i, line in enumerate(infile):
    p0, p1 = (tuple(map(int, p.split(","))) for p in line.strip().split("~"))
    cubes[i] = (p0, p1)

gminx = min(min(x0, x1) for ((x0, _, _), (x1, _, _)) in cubes.values())
gminy = min(min(y0, y1) for ((_, y0, _), (_, y1, _)) in cubes.values())
gmaxx = max(max(x0, x1) for ((x0, _, _), (x1, _, _)) in cubes.values())
gmaxy = max(max(y0, y1) for ((_, y0, _), (_, y1, _)) in cubes.values())

grid = dict(((x,y,0), -1) for x in range(gminx, gmaxx+1) for y in range(gminy, gmaxy+1))

# lower cubes starting from the lowest ones, stopping when they collide with something
to_lower = list(range(len(cubes)))
to_lower.sort(key = lambda i: min(cubes[i][0][2], cubes[i][1][2]))
resting_on = {}
while to_lower:

    c_i = to_lower.pop(0)
    c = cubes[c_i]

    rest = set()
    rest_z = -1

    cminx = min(c[0][0], c[1][0])
    cminy = min(c[0][1], c[1][1])
    cminz = min(c[0][2], c[1][2])
    cmaxx = max(c[0][0], c[1][0])
    cmaxy = max(c[0][1], c[1][1])
    cmaxz = max(c[0][2], c[1][2])

    for ((gx, gy, gz), gi) in grid.items():
        if gx >= cminx and gx <= cmaxx and gy >= cminy and gy <= cmaxy:
            if gz > rest_z:
                rest_z = gz
                rest = set([gi])
            elif gz == rest_z:
                rest.add(gi)

    print(f"{c_i} resting on {rest}")
    resting_on[c_i] = rest

    new_z = rest_z + 1
    delta_z = new_z - cminz
    c = ((c[0][0], c[0][1], c[0][2] + delta_z),
         (c[1][0], c[1][1], c[1][2] + delta_z))
    cminz = min(c[0][2], c[1][2])
    cmaxz = max(c[0][2], c[1][2])

    for z in range(cminz, cmaxz+1):
        for y in range(cminy, cmaxy+1):
            for x in range(cminx, cmaxx+1):
                assert((x, y, z) not in grid)
                grid[(x, y, z)] = c_i

total = 0
can_disintegrate = set(range(len(cubes)))
for ri, rs in resting_on.items():
    if len(rs) == 1:
        r, = rs
        can_disintegrate.discard(r)

for c_i in range(len(cubes)):
    if c_i in can_disintegrate:
        continue
    fall = set()
    new_fall = set([c_i])
    while len(new_fall) > len(fall):
        fall.update(new_fall)
        for c_j in range(len(cubes)):
            if all(ro in new_fall for ro in resting_on[c_j]):
                new_fall.add(c_j)
    total += len(fall) - 1

print(total)

158 resting on {-1}
197 resting on {-1}
288 resting on {-1}
649 resting on {-1}
655 resting on {-1}
1018 resting on {-1}
1215 resting on {-1}
1345 resting on {-1}
295 resting on {-1}
429 resting on {1215}
506 resting on {-1}
1122 resting on {-1}
1140 resting on {-1}
1201 resting on {-1}
1274 resting on {-1}
1314 resting on {197}
5 resting on {288}
77 resting on {1122, 655}
79 resting on {295}
215 resting on {-1}
247 resting on {-1}
251 resting on {429}
460 resting on {-1}
659 resting on {-1}
853 resting on {1274}
889 resting on {-1}
1054 resting on {-1}
134 resting on {1201}
555 resting on {1054}
590 resting on {288}
752 resting on {1018}
1028 resting on {853}
4 resting on {247, 655}
160 resting on {429}
170 resting on {649}
175 resting on {659}
200 resting on {1314}
478 resting on {555}
589 resting on {429}
815 resting on {1201, 1140}
852 resting on {506}
1001 resting on {853}
1145 resting on {251}
876 resting on {134}
1212 resting on {200, 79}
1213 resting on {1028}
234 resting on {1